# Scryfall Stuff

In [1]:
import requests
import json
import numpy as np



In [5]:
res  = requests.get('https://api.scryfall.com/sets')
response = json.loads(res.text)

In [ ]:
for set in response["data"]:
    if set["set_type"] == "expansion":
        print(f"Set Name: {set["name"]}")
        print(f'Set Code: {set["code"]}')
        print('#'*50)

In [7]:
response['data'][0]

{'object': 'set',
 'id': '98ffb0ad-639e-4861-b81d-dd5ae734badf',
 'code': 'tblb',
 'name': 'Bloomburrow Tokens',
 'uri': 'https://api.scryfall.com/sets/98ffb0ad-639e-4861-b81d-dd5ae734badf',
 'scryfall_uri': 'https://scryfall.com/sets/tblb',
 'search_uri': 'https://api.scryfall.com/cards/search?include_extras=true&include_variations=true&order=set&q=e%3Atblb&unique=prints',
 'released_at': '2024-08-02',
 'set_type': 'token',
 'card_count': 1,
 'parent_set_code': 'blb',
 'digital': False,
 'nonfoil_only': False,
 'foil_only': False,
 'icon_svg_uri': 'https://svgs.scryfall.io/sets/blb.svg?1711944000'}

### Retrieve a set

In [9]:
res  = requests.get('https://api.scryfall.com/cards/search?q=set%3Dxln')
response = json.loads(res.text)

In [10]:
res2 = requests.get(response['next_page'])
response2 = json.loads(res2.text)

In [16]:
type(response['data'])

list

# SQLite Stuff

### Initial Table

In [7]:
import sqlite3
import pandas as pd
import numpy as np

In [3]:
connection = sqlite3.connect('./Data/LSRDB.db')

In [4]:

set_df = pd.read_sql_query(f"SELECT * FROM blb",connection)


In [15]:
def type_selection(type_line):
    if type_line.lower().find('creature')>=0:
        return 'creature'
    elif type_line.lower().find('sorcery')>=0:
        return 'sorcery'
    elif type_line.lower().find('instant')>=0:
        return 'instant'
    elif type_line.lower().find('enchantment')>=0:
        return 'enchantment'
    elif type_line.lower().find('artifact')>=0:
        return 'artifact'
    elif type_line.lower().find('land')>=0:
        return 'land'
    elif type_line.lower().find('planeswalker')>=0:
        return 'planeswalker'
    else:
        return 'other'

In [16]:
set_df['basic_type'] = set_df['type_line_1'].apply(type_selection)

In [17]:
set_df['basic_type'].value_counts()

basic_type
creature        163
instant          36
sorcery          29
enchantment      21
land             17
artifact         12
planeswalker      1
Name: count, dtype: int64

In [14]:
set_df[set_df['basic_type']=='other']

,scryfall_id,oracle_id,name,layout,image_uri_1,image_uri_2,mana_cost_1,mana_cost_2,cmc,type_line_1,...,colors_1,colors_2,color_identity,keywords,set_id,set_name,collector_number,rarity,booster,basic_type
180,acfde780-899a-4c5b-a39b-f4a3ff129103,3c970f43-17da-4187-a084-efd53b579067,"Ral, Crackling Wit",normal,https://cards.scryfall.io/normal/front/a/c/acf...,NA,{2}{U}{R},NA,4,Legendary Planeswalker — Ral,...,"['R', 'U']",NA,"['R', 'U']",[],a2f58272-bba6-439d-871e-7a46686ac018,Bloomburrow,230,mythic,True,other


In [3]:
cursor = connection.cursor()

In [3]:
connection.total_changes

0

In [4]:
cursor.execute('CREATE TABLE IF NOT EXISTS mtgSets (scryfall_id TEXT, code TEXT, name TEXT, released_at DATE, set_type TEXT, card_count INTEGER, UNIQUE(scryfall_id))')

In [10]:
connection.commit()

In [1]:
from utils.sets import sets_list
sets_list()

In [ ]:
for set in response['data']:
    if set['set_type'] == 'expansion':
        print(f'id: {set["id"]}')
        print(f'code: {set["code"]}')
        print(f'name: {set["name"]}')
        print(f'released_at: {set["released_at"]}')
        print(f'set_type: {set["set_type"]}')
        print(f'card_count: {set["card_count"]}')
        

In [14]:
connection.commit()

In [5]:
cursor.execute('''
               CREATE TABLE IF NOT EXISTS xln (
                    scryfall_id TEXT,
                    oracle_id TEXT,
                    name TEXT,
                    layout TEXT, 
                    image_uri_1 TEXT, 
                    image_uri_2 TEXT, 
                    mana_cost_1 TEXT, 
                    mana_cost_2 TEXT, 
                    cmc INTEGER,
                    type_line_1 TEXT,
                    type_line_2 TEXT,
                    oracle_text_1 TEXT,
                    oracle_text_2 TEXT,
                    power_1 INTEGER,
                    power_2 INTEGER,
                    toughness_1 INTEGER,
                    toughness_2 INTEGER,
                    colors_1 TEXT,
                    colors_2 TEXT,
                    color_identity TEXT,
                    keywords TEXT,
                    set_id TEXT,
                    set_name TEXT,
                    collector_number,
                    rarity TEXT,
                    booster BOOLEAN
               )
               ''')

In [13]:
for card in response['data']:

    # print(json.dumps(card, sort_keys=True, indent=4))
    # print("#"*100)

    pows = {}
    tous = {}

    if 'power' in list(card.keys()):
        pows['pow_1'] = card['power']
        pows['pow_2']  = 999
    elif 'card_faces' in list(card.keys()):        
        count = 1
        for face in card['card_faces']:
            if 'power' in list(face.keys()):
                pows[f'pow_{count}'] = face['power']
            else:
                pows[f'pow_{count}'] = 999
            count+=1
    else:
        pows['pow_1'] = 999
        pows['pow_2'] = 999
    
    if 'toughness' in list(card.keys()):
        tous['tou_1'] = card['toughness']
        tous['tou_2']  = 999
    elif 'card_faces' in list(card.keys()):        
        count = 1
        for face in card['card_faces']:
            if 'toughness' in list(face.keys()):
                tous[f'tou_{count}'] = face['toughness']
            else:
                tous[f'tou_{count}'] = 999
            count+=1
    else:
        tous['tou_1']  = 999
        tous['tou_2']  = 999

    try:
        card_text_1 = card['oracle_text'].replace('"','').replace("'",'')
        card_text_2 = "NA"
    except:
        card_text = [face['oracle_text'] for face in card['card_faces']]

        card_text_1 = card_text[0].replace('"','').replace("'",'')
        card_text_2 = card_text[1].replace('"','').replace("'",'')

    try:
        image_uri_1 = card['image_uris']['normal']
        image_uri_2 = "NA"
    except:
        image_uris = [face['image_uris']['normal'] for face in card['card_faces']]

        image_uri_1 = image_uris[0]
        image_uri_2 = image_uris[1]

    try:
        mana_cost_1 = card['mana_cost']
        mana_cost_2 = "NA"
    except:
        mana_costs = [face['mana_cost'] for face in card['card_faces']]

        mana_cost_1 = mana_costs[0]
        mana_cost_2 = mana_costs[1]

    try:        
        type_1 = card['type_line']
        type_2 = "NA"
    except:
        types = [face['type_line'] for face in card['card_faces']]

        type_1 = types[0]
        type_2 = types[1]

    try:        
        colors_1 = card['colors']
        colors_2 = "NA"
    except:
        colorss = [face['colors'] for face in card['card_faces']]

        colors_1 = colorss[0]
        colors_2 = colorss[1]
        

    cursor.execute(f'''
                        INSERT INTO xln VALUES (
                            '{card['id']}',
                            '{card['oracle_id']}',
                            "{card['name'].replace('"','').replace("'",'')}",
                            '{card['layout']}',
                            '{image_uri_1}',
                            '{image_uri_2}',
                            '{mana_cost_1}',
                            '{mana_cost_2}',
                            {card['cmc']},
                            '{type_1}',
                            '{type_2}',
                            '{card_text_1}',
                            '{card_text_2}',
                            {pows['pow_1']},
                            {pows['pow_2']},
                            {tous['tou_1']},
                            {tous['tou_2']},
                            "{colors_1}",
                            "{colors_2}",
                            "{card['color_identity']}",
                            "{card['keywords']}",
                            '{card['set_id']}',
                            '{card['set_name']}',
                            '{card['collector_number']}',
                            '{card['rarity']}',
                            '{card['booster']}'
                            )
                    ''')

{
    "arena_id": 65961,
    "artist": "Anna Steinbauer",
    "artist_ids": [
        "3516496c-c279-4b56-8239-720683d03ae0"
    ],
    "booster": true,
    "border_color": "black",
    "card_back_id": "0aeebaf5-8c7d-4636-9e82-8c27447861f7",
    "cardmarket_id": 301689,
    "cmc": 2.0,
    "collector_number": "1",
    "color_identity": [
        "W"
    ],
    "colors": [
        "W"
    ],
    "digital": false,
    "edhrec_rank": 11935,
    "finishes": [
        "nonfoil",
        "foil"
    ],
    "foil": true,
    "frame": "2015",
    "full_art": false,
    "games": [
        "arena",
        "paper",
        "mtgo"
    ],
    "highres_image": true,
    "id": "21c950d7-b4f6-4902-8c9a-98f2933f9fa5",
    "illustration_id": "9e1bc3e7-2541-4a83-842a-70550a260f5e",
    "image_status": "highres_scan",
    "image_uris": {
        "art_crop": "https://cards.scryfall.io/art_crop/front/2/1/21c950d7-b4f6-4902-8c9a-98f2933f9fa5.jpg?1562552020",
        "border_crop": "https://cards.scryfall.io/

['object',
 'id',
 'oracle_id',
 'multiverse_ids',
 'mtgo_id',
 'mtgo_foil_id',
 'arena_id',
 'tcgplayer_id',
 'cardmarket_id',
 'name',
 'lang',
 'released_at',
 'uri',
 'scryfall_uri',
 'layout',
 'highres_image',
 'image_status',
 'image_uris',
 'mana_cost',
 'cmc',
 'type_line',
 'oracle_text',
 'power',
 'toughness',
 'colors',
 'color_identity',
 'keywords',
 'legalities',
 'games',
 'reserved',
 'foil',
 'nonfoil',
 'finishes',
 'oversized',
 'promo',
 'reprint',
 'variation',
 'set_id',
 'set',
 'set_name',
 'set_type',
 'set_uri',
 'set_search_uri',
 'scryfall_set_uri',
 'rulings_uri',
 'prints_search_uri',
 'collector_number',
 'digital',
 'rarity',
 'card_back_id',
 'artist',
 'artist_ids',
 'illustration_id',
 'border_color',
 'frame',
 'full_art',
 'textless',
 'booster',
 'story_spotlight',
 'edhrec_rank',
 'penny_rank',
 'prices',
 'related_uris',
 'purchase_uris']

In [1]:
from utils.sets import set_cards

set_cards('xln')